## Deep Learning for VaR Predictions in the UK Residential Real Estate Market
***

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime
from tqdm import tqdm
import time
import random

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import shapiro, chi2
from arch import arch_model

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Flatten, SimpleRNN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping


from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_squared_error

import sys
import vartests

from hpi_data import HPI_data


import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=ConvergenceWarning)

matplotlib.style.use('seaborn-paper')
plt.rcParams['savefig.facecolor']='white'
plt.rcParams["figure.figsize"] = (18,10)

***
## Data Preprocessing & EDA
### [House Price Index Data](https://www.gov.uk/government/collections/uk-house-price-index-reports)

In [ ]:
hpi = HPI_data('UK-HPI-full-file-2023-03.csv',
             'Ward_to_Local_Authority_District_to_County_to_Region_to_Country_Lookup_in_United_Kingdom.csv')

average_price_df, detached_price_df, semidet_price_df, terraced_price_df, flat_price_df = hpi.get_data().get_property_dfs()

### Price Plot

In [ ]:
df_list_str = ['Average Price', 'Average Detached Price', 'Average Semi-Detached Price',
               'Average Terraced Price', 'Average Flat Price']
n = 0
for df in [average_price_df, detached_price_df, semidet_price_df, terraced_price_df, flat_price_df]:
    fig, ax = plt.subplots()

    for column in df:
        ax.plot(df.index, df[column], marker='', color='#5A5A5A', linewidth=1, alpha=0.6)
    ax.plot(df.index, df['England'], marker='', color='red', linewidth=3, alpha=0.8)
    ax.plot(df.index, df['London'], marker='', color='#035962', linewidth=3, alpha=0.8)

    ax.set_xlim(df.index.min()+dt.timedelta(days=-365),df.index.max()+dt.timedelta(days=1550))
    num=0
    for i in df.values[-1]:
        name=list(df)[num]
        num+=1
        if name != 'England' and name != 'London':
            if name == "Yorkshire and The Humber":
                ax.text(df.index.max()+dt.timedelta(days=30), i-5000, name,
                        horizontalalignment='left', size='small', color='#5A5A5A')
            else:
                ax.text(df.index.max()+dt.timedelta(days=30), i, name,
                        horizontalalignment='left', size='small', color='#5A5A5A')

    cur_england = df['England'].tail(1)
    cur_london = df['London'].tail(1)

    # And add a special annotation for the group we are interested in
    ax.text(df.index.max()+dt.timedelta(days=60), cur_england,
            'England (%s)' % (f'£{int(cur_england/1000):,}k'), horizontalalignment='left', color='red')
    ax.text(df.index.max()+dt.timedelta(days=60), cur_london,
            'London (%s)' % (f'£{int(cur_london/1000):,}k'), horizontalalignment='left', color='#035962')

    y_labels = [f'£{int(amt/1000):,}k' for amt in ax.get_yticks()]
    ax.set_yticklabels(y_labels)
    sns.despine()

    plt.xlabel('Date', fontweight='bold')
    plt.ylabel(df_list_str[n], fontweight='bold')
    plt.savefig("plots/" + df_list_str[n].lower().replace(" ", "_") + "_lineplot.png")
    plt.show()
    
    n += 1

### Returns Dist Plot

In [ ]:
average_price_regions_df = average_price_df.loc[:, average_price_df.columns != "England"] 
average_price_country_df = average_price_df["England"]

for i, column in enumerate(average_price_regions_df.columns, 1):
    dist = np.log1p(average_price_regions_df[column].pct_change().dropna())
    plt.subplot(3,3,i)
    plt.xlim(-0.1,0.1)
    plt.xticks([-0.1, -0.05, 0.0, 0.05, 0.1], [-0.1, -0.05, 0.0, 0.05, 0.1])
    plt.xlabel(" ")
    if i == 8:
        plt.xlabel("Log Returns", fontsize=15)
    plt.ylim(0,70)
    plt.yticks([20, 40, 60], [20, 40, 60])
    plt.ylabel(" ")
    if i == 4:
        plt.ylabel("Frequency", fontsize=15)
    plt.text(0.049, 65, 'p = {:0.2e}'.format(shapiro(dist.reset_index()[column])[1]), fontweight="bold")
    plt.title(column, fontweight="bold")
    sns.histplot(dist, color='#035962', edgecolor="w", alpha=1)
sns.despine()
plt.savefig("plots/" + 'average_price_regional_distributions.png')
plt.show()

In [ ]:
dist = np.log1p(average_price_country_df.pct_change().dropna())
plt.xlim(-0.07,0.07)
plt.xticks([-0.05, -0.025, 0.0, 0.025, 0.05], [-0.05, -0.025, 0.0, 0.025, 0.05])
plt.xlabel("Log Returns", fontsize=15)

plt.ylim(0,70)
plt.yticks([20, 40, 60], [20, 40, 60])
plt.ylabel("Frequency", fontsize=15)
plt.text(0.059, 67, 'p = {:0.2e}'.format(shapiro(dist.reset_index()['England'])[1]), fontweight="bold")
plt.title('England', fontweight="bold")
sns.histplot(dist, color='#035962', edgecolor="w", alpha=1)
sns.despine()
plt.savefig("plots/" + 'average_price_national_distribution.png')
plt.show()

### Count Nulls

In [ ]:
for df in [average_price_df, detached_price_df, semidet_price_df, terraced_price_df, flat_price_df]:
    print(dict(average_price_df.isnull().sum()))

### Create Returns DataFrames

In [ ]:
average_returns_df, detached_returns_df, semidet_returns_df, terraced_price_df, flat_returns_df = HPI_data.get_hpi_returns([average_price_df, detached_price_df, semidet_price_df, terraced_price_df, flat_price_df])

### Plotting Average Price Returns

In [ ]:
average_returns_df_plot = average_returns_df.loc[:, average_returns_df.columns != "England"]
for i, column in enumerate(average_returns_df_plot.columns, 1):
    plt.subplot(3,3,i)
    average_returns_df[column].plot(color='#035962')
    plt.xlabel(" ")
    plt.ylim(-0.085,0.085)
    if i == 8:
        plt.xlabel("Date", fontsize=15)
    #plt.ylim(0,70)
    #plt.yticks([20, 40, 60], [20, 40, 60])
    plt.ylabel(" ")
    if i == 4:
        plt.ylabel("Returns", fontsize=15)
    plt.title(column, fontweight="bold")

sns.despine()
plt.savefig("plots/" + 'average_price_returns.png')
plt.show()

***
## Value-at-Risk

### Filtered Historic Simulation (GJR-GARCH)

In [ ]:
def garch_model(returns, vol="Garch", p=1, o=1, q=1, dist="normal"):
    return arch_model(returns, vol=vol, p=p, o=o, q=q, dist=dist)


def FHS_VaR(returns, col, garch_model):
    
    first_obs = returns.index[0]
    last_obs = returns.index[-2]
    start_fc = returns.index[-1]
    
    res = garch_model.fit(disp='off', last_obs=last_obs)
    
    forecasts = res.forecast(start=start_fc, reindex=False, horizon=1)
    cond_mean = forecasts.mean[start_fc:]
    cond_var = forecasts.variance[start_fc:]
    
    std_rets = ((returns[:last_obs][col] - res.params["mu"])) / res.conditional_volatility
    std_rets = std_rets.dropna()
    q = std_rets.quantile([0.05])
    
    value_at_risk = -(cond_mean.values - np.sqrt(cond_var).values * q.values[None, :])
    value_at_risk = pd.DataFrame(value_at_risk, columns=["VaR 95%"], index=cond_var.index)    
    
    value_at_risk['actual_return'] = returns[-1:]
    
    return value_at_risk

### Generate VaR Observations

In [ ]:
#@ignore_warnings(category=ConvergenceWarning)
def generate_VaR_observations(prices, column, prop, model_window, no_observations):
    
    df = pd.DataFrame(prices[column])
    returns_ls = []
    VaRs = []
    
    if not no_observations:
        no_obs = len(df)-model_window-1
    else:
        no_obs = no_observations
    
    for i in tqdm(range(no_obs), desc="Loading…",ascii=False, ncols=75, position=0, leave=True):
        if no_observations == False:
            period = df.iloc[i:i+model_window, :]
        else:
            nrows = range(df.shape[0])
            ix = random.randint(nrows.start, nrows.stop-model_window-1)
            period = df.iloc[ix:ix+model_window, :]       
        scaler = StandardScaler()
        returns = pd.DataFrame(scaler.fit_transform(period), columns=period.columns, index=period.index)
        gm = garch_model(returns)
        VaRs.append(FHS_VaR(returns[int((model_window-1)/2+1):], column, gm))
        returns_ls.append(returns[int((model_window-1)/2+1):].values)
    VaRs = pd.concat(VaRs)
    
    x = []
    for r in returns_ls:
        x.append(np.array(r.flatten()))
    VaRs['x'] = x
    VaRs = VaRs.sort_index()
    VaRs = VaRs.reset_index()
    
    VaRs['region'] = column
    VaRs['property']   = prop

    return VaRs

#### Initiate VaR DataFrame Lists (Filter Regions)

In [ ]:
average_VaRs, detached_VaRs, semidet_VaRs, terraced_VaRs, flat_VaRs = [], [], [], [], []

regions = ['England', 'London', 'South West', 'West Midlands', 'Yorkshire and The Humber']

In [ ]:
for region in regions:

    print('Generating VaR Observations for %s:' % region)
    time.sleep(1)
    
    average_VaRs.append(generate_VaR_observations(average_returns_df, region, 'average', 49, False))
    detached_VaRs.append(generate_VaR_observations(detached_returns_df, region, 'detached', 49, False))
    semidet_VaRs.append(generate_VaR_observations(semidet_returns_df, region, 'semi_detached', 49, False))
    terraced_VaRs.append(generate_VaR_observations(terraced_returns_df, region, 'terraced', 49, False))
    flat_VaRs.append(generate_VaR_observations(flat_returns_df, region, 'flat', 49, False))

In [ ]:
for prop in [average_VaRs, detached_VaRs, semidet_VaRs, terraced_VaRs, flat_VaRs]:
    for region in prop:
        region = region.reset_index()

### Combine VaR DataFrames

In [ ]:
full_df = pd.concat([average_VaRs[0], detached_VaRs[0], semidet_VaRs[0], terraced_VaRs[0], flat_VaRs[0],
                average_VaRs[1], detached_VaRs[1], semidet_VaRs[1], terraced_VaRs[1], flat_VaRs[1],
                average_VaRs[2], detached_VaRs[2], semidet_VaRs[2], terraced_VaRs[2], flat_VaRs[2],
                average_VaRs[3], detached_VaRs[3], semidet_VaRs[3], terraced_VaRs[3], flat_VaRs[3],
                average_VaRs[4], detached_VaRs[4], semidet_VaRs[4], terraced_VaRs[4], flat_VaRs[4]], ignore_index=True)

### VaR Plot Function

In [ ]:
def plot_VaR(df, 
             VaR_col=["VaR 95%", "Predictions"],
             return_col = "actual_return",
             VaR_colour = ["#035962", "#8B0000"],
             markers = {"#000000": "o", "red": "x", "#00C500": "x", "#EC91D8": "x"},
             labels = {"#000000": "No Breach", "red": "Predicted breach", "#00C500": "Correct Breach", "#EC91D8": "Prevented breach"},
             sizes = {"#000000": 18, "red": 80, "#00C500": 80, "#EC91D8": 80},
             single = False,
             save_name=None,
             print_plot=False):
    
    
    if len(VaR_col) == 2:
        df = df[[return_col, VaR_col[0], VaR_col[1]]]
        df = df.groupby(df.index).mean()
        #df = df.reset_index()
        
        ax = df.plot(y=VaR_col, legend=False, linestyle = "--",
                     linewidth = 1.5, style=VaR_colour)
        

        df = df.sort_index()
        xlim = ax.set_xlim(df.index[0], df.index[-1])
        ylim = ax.set_ylim(df.min(numeric_only=True).min()*1.025,
                           df.max(numeric_only=True).max()*1.025)

        point_colours = []
        for i, x in df.iterrows():
            if x[return_col] > x[VaR_col[0]]:
                if x[return_col] > x[VaR_col[1]]:
                    point_colours.append(list(markers.keys())[0])
                else:
                    point_colours.append(list(markers.keys())[1])
            else:
                if x[return_col] <= x[VaR_col[1]]:
                    point_colours.append(list(markers.keys())[2])
                else:
                    point_colours.append(list(markers.keys())[3])

        point_colours = np.array(point_colours, dtype="object")
        for colour in np.unique(point_colours):
            
            sel = point_colours == colour
            if sel.sum() > 1:

                ax.scatter(
                    x=df.index[sel],
                    y=df[return_col][sel],
                    marker=markers[colour],
                    c=point_colours[sel],
                    label=labels[colour],
                    s=sizes[colour])
            plt.axhline(y=0, color='r', linestyle='--', alpha=0.4)
            ax.set_xlabel('Date', fontweight='bold')
            ax.set_ylabel('Returns', fontweight='bold')
            leg = ax.legend(frameon=True, ncol=3, loc='upper left', fontsize=11)

    
        
        if save_name != None:
            plt.savefig('%s.png' % save_name)
        
        if print_plot==True:
            plt.show()
            
        plt.close()
    
    else:

        df = df[["Date", return_col, VaR_col]]

        df = df.groupby(['Date']).mean()
        df = df.reset_index()


        ax = df.plot(x="Date",
                      y=VaR_col,
                      legend=False,
                      linestyle = "--",
                      linewidth = 1.5,
                      style=VaR_colour)

        df = df.sort_values('Date')
        xlim = ax.set_xlim(df['Date'].min(), df['Date'].max())
        ylim = ax.set_ylim(df.min(numeric_only=True).min()*1.025,
                           0)
                           #df.max(numeric_only=True).max()*1.025)

        point_colours = []
        for i, x in df.iterrows():
            if x[return_col] > x[VaR_col]:
                point_colours.append(list(markers.keys())[0])
            else:
                point_colours.append(list(markers.keys())[1])

        point_colours = np.array(point_colours, dtype="object")

        for colour in np.unique(point_colours):

            sel = point_colours == colour
            if sel.sum() > 1:

                ax.scatter(
                    df['Date'].iloc[sel],#.index[sel],
                    df[return_col].iloc[sel],
                    marker=markers[colour],
                    c=point_colours[sel],
                    label=labels[colour],
                    s=sizes[colour])
            plt.axhline(y=0, color='r', linestyle='--', alpha=0.4)
            ax.set_xlabel('Date', fontweight='bold')
            ax.set_ylabel('Returns', fontweight='bold')
            leg = ax.legend(frameon=True, ncol=3, loc='upper left')
  

        if save_name != None:
            plt.savefig("plots/" + '%s.png' % save_name)

        if print_plot==True:
            plt.show()
            
        plt.close()

***
## Neural Networks

### ANN, RNN and RNN-LSTM Functions

In [ ]:
def build_ann(hidden_units=64, learning_rate=0.001, num_hidden_layers=2, drop_out=0.1):
    
    model = keras.Sequential() # initiate model
    
    model.add(Flatten(input_shape=(24,))) # first layer
    
    # add hidden layers
    for _ in range(num_hidden_layers-1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(drop_out))

    model.add(Dense(1)) # output layer
    
    # compile (optimiser=Adam, loss=MSE, metric=MAE) with given learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error'])
    return model

def build_rnn(hidden_units=64, learning_rate=0.001, num_hidden_layers=2, drop_out=0.1):
    
    model = keras.Sequential()  # initiate model
    
    # first RNN layer
    model.add(SimpleRNN(units=hidden_units, activation = "relu", 
                                     return_sequences = True, input_shape = (24,1, )))
    model.add(Dropout(drop_out)) # add dropout
    
    # add hidden layers
    if num_hidden_layers > 2:
        for _ in range(num_hidden_layers-2):
            model.add(SimpleRNN(units=hidden_units, activation='relu', return_sequences=True))
            model.add(Dropout(drop_out))        
    model.add(SimpleRNN(units=hidden_units, activation='relu'))
    
    model.add(Dense(1)) # output layer
    
    # compile (optimiser=Adam, loss=MSE, metric=MAE) with given learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error']) 
    return model

def build_lstm(hidden_units=64, learning_rate=0.001, num_hidden_layers=2, drop_out=0.1):
    
    model = keras.Sequential() # initiate model
    
    # Add LSTM layer
    model.add(LSTM(units=hidden_units, return_sequences=True, input_shape=(24,1,)))
    
    model.add(Dropout(drop_out)) # add drop out
    
    # add hidden layers
    if num_hidden_layers > 2:
        for _ in range(num_hidden_layers-2):
            model.add(SimpleRNN(units=hidden_units, activation='relu', return_sequences=True))
            model.add(Dropout(drop_out))
    model.add(SimpleRNN(units=hidden_units, activation='relu'))
    
    model.add(Dense(1)) # output layer
    
    # compile (optimiser=Adam, loss=MSE, metric=MAE) with given learning rate
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='mean_squared_error',
                  metrics=['mean_absolute_error']) 
    return model

### Custom Train Test Function

In [ ]:
def train_test(df, date_col='Date', region="England", prop="average", x_col='x', y_col='1%',
               random=False, test_size=0.2, seed=42):
    
    if random:
        x_train, x_test, y_train, y_test = train_test_split(df[x_col], df[y_col],
                                                            test_size=test_size, random_state=seed)
    else:
        train = df[(df['region']!=region) & (df['property']!=prop)]
        test  = df[(df['region']==region) & (df['property']==prop)]

        #df.query('20130101 < date < 20130201')
        x_train = train[x_col]
        x_test  = test[x_col]
        y_train = train[y_col]
        y_test  = test[y_col]
        
    #print(x_train.shape, x_test.shape, y_train.shape, y_test.shape )
    return x_train, x_test, y_train, y_test

### Run NN Modelling Function

In [ ]:
def run_optimisation(x_train, y_train, param_grid, nn_func=build_ann, epochs=100, bs=64):
    
    # initiate early stopping
    early_stopping = EarlyStopping(monitor='mean_absolute_error', patience=6)
    
    # use tensorflow scikit learn wrapper for Keras Regressor
    # pass NN model, epchs, batch size and early stopping callback
    model = KerasRegressor(build_fn=nn_func, epochs=epochs, batch_size=bs, verbose=0, callbacks=[early_stopping])
    
    # Initiate pipeline: pass model wrapper and scaler function
    pipe = Pipeline([('scale', StandardScaler()),('model', model)])
    
    # pass pipe to TransformedTargetRegressor wrapper and standard scaler transformer
    # this is to utilise scaling of y values
    estimator = TransformedTargetRegressor(regressor=pipe, transformer=StandardScaler())
    
    # Initiate RandomizedSearchCV (3-Fold, 5-Iterations)
    grid = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid, cv=3,
                        verbose=1, scoring='neg_mean_squared_error', n_iter = 5)
    
    # Fit and run RandomizedSearchCV
    grid_result = grid.fit(x_train, y_train)
    
    #return result of model training
    return grid_result

### Loss Plot Function

In [ ]:
def plot_optimal_model_loss(grid_result, print_plot=False, plot_name=None):
    
    hist = grid_result.best_estimator_.regressor_.named_steps['model'].model.history
    
    mse  = hist.history['loss']
    rmse = np.sqrt(mse)
    mae  = hist.history['mean_absolute_error']

    epoch_count = range(1, len(mse) + 1)

    # Visualize loss history
    plt.plot(epoch_count, rmse, 'b-.')
    plt.plot(epoch_count, mse, 'r--')
    plt.plot(epoch_count, mae, '#035962')
    
    plt.legend(['RMSE', 'MSE', 'MAE'], fontsize=13)
    plt.xlabel('Epoch', fontweight='bold', fontsize=13)
    plt.ylabel('Loss', fontweight='bold', fontsize=13)
    
    plt.text(epoch_count[-1]+0.2, rmse[-1], round(rmse[-1], 2), bbox=dict(facecolor='white', edgecolor='b'))
    plt.text(epoch_count[-1]+0.2, mse[-1], round(mse[-1], 2), bbox=dict(facecolor='white', edgecolor='r'))
    plt.text(epoch_count[-1]+0.2, mae[-1], round(mae[-1], 2), bbox=dict(facecolor='white', edgecolor='#035962'))
    
    sns.despine()
    
    if plot_name != None:
        plt.savefig("%s.png" % plot_name)
    
    if print_plot:
        plt.show()
        
    plt.close()

### Evaluate Model Function

In [ ]:
def best_model_evaluation(model, test_df, test_x, actual_col, return_col, print_plot=False, plot_name=None):
    
    # retrieve optimal model from RandomizedSearchCV
    optim_model = model.best_estimator_

    # make predictions
    predictions = optim_model.predict(np.array(test_x.tolist()).astype('float64'))
    
    eval_df = pd.DataFrame(test_df) #get test dataframe
    eval_df['Predictions'] = predictions # assign predictions to column
    eval_df = eval_df.groupby(eval_df['Date']).mean() # group by date
    
    # calculate breach percentage of VaR and predicted VaR
    pred_backtest = round(((eval_df['Predictions'] > eval_df[return_col]).sum() / len(eval_df))*100, 2)
    actual_backtest = round(((eval_df[actual_col] > eval_df[return_col]).sum() / len(eval_df))*100, 2)
    
    # calculate RMSE
    rmse = round(np.sqrt(mean_squared_error(y_pred=eval_df['Predictions'], y_true=eval_df[actual_col])),3)
    
    # Plot VaR plot using function
    plot_VaR(eval_df, save_name=plot_name, print_plot=print_plot)
    
    #Return Violations as binary output for Act and Pred VaR
    eval_df['actual_violation'] = eval_df.apply(lambda x: 1 if x[actual_col] >= x[return_col] else 0, axis=1)
    eval_df['pred_violation'] = eval_df.apply(lambda x: 1 if x['Predictions'] >= x[return_col] else 0, axis=1)
    
    # Use this to calculate Kupiec test (1 dof) for both
    actl_kup = vartests.kupiec_test(eval_df['actual_violation'], var_conf_level=0.95, conf_level=0.95)['log-likelihood']
    pred_kup = vartests.kupiec_test(eval_df['pred_violation'], var_conf_level=0.95, conf_level=0.95)['log-likelihood']
    
    # get p-value from kupiec test
    actl_kup, pred_kup = round(chi2.pdf(actl_kup, 1), 4), round(chi2.pdf(pred_kup, 1), 4)
    
    # return results for final results summary dataframe
    return [rmse, actual_backtest, pred_backtest, actl_kup, pred_kup, model.best_params_]

***
## Running NN Modelling

### Hyperparameter Grid and Labels 

In [ ]:
param_grid = {'regressor__model__hidden_units'      :    [32, 64, 128],
              'regressor__model__learning_rate'     :     [1e-2, 1e-3],
              'regressor__model__num_hidden_layers' :           [3, 4],
              'regressor__model__drop_out'          : [0.05, 0.1, 0.2]} 


properties = ['average', 'detached', 'semi_detached', 'terraced', 'flat']

model_names = ['ANN', 'RNN', 'LSTM']

### Run Modelling for each Region & Property Type

In [ ]:
final_results = []
for region in regions:
    for prop in properties:
        n = 0
        for model_func in [build_ann, build_rnn, build_lstm]:
            file_region = "%s_%s" % (prop, region.replace(" ", "_"))
            file_model  = model_names[n]
            
            print('~~~~~~~~~~~~~~~~~~~~ %s - %s - %s ~~~~~~~~~~~~~~~~~~~~' % (model_names[n], prop, region))

            x_train, x_test, y_train, y_test = train_test(full_df, region=region, prop=prop, y_col='VaR 95%')
            
            # run optimisation
            cv_results = run_optimisation(np.array(x_train.tolist()).astype('float64'), np.array(
                y_train.values).astype('float64'), param_grid, nn_func=model_func, epochs=200, bs=128)

            # plot best model loss plot from cv results above - save plot to folder
            plot_optimal_model_loss(cv_results, print_plot=False,
                plot_name= '%s_%s_optimal_model_loss_plt' % (file_region, file_model))

            # make predictions on test data, return performance metrics - save backtest plot to folder
            results = best_model_evaluation(cv_results, full_df.loc[x_test.index], x_test,
                'VaR 95%', 'actual_return', print_plot=True, 
                plot_name= '%s_%s_var_prediction_backtest_plot' % (file_region, file_model))

            # append model stats to list for later evaluation
            final_results.append([prop, region, model_names[n],
                                    results[0], results[1], results[2], results[3],
                                    results[4], results[5]])

            n += 1

# write output of model predictions evaluations to dataframe
final_results = pd.DataFrame(final_results, columns=['property', 'region', 'model',
                                                          'rmse', 'btest_actual', 'btest_predict',
                                                          'act_pval', 'pred_pval',
                                                          'best_params'])

# explode dict of best params into seperate columns
final_results = pd.concat([final_results.drop(['best_params'], axis=1),
                              final_results['best_params'].apply(pd.Series)], axis=1)
final_results.columns = [x.replace('regressor__model__', '') for x in final_results.columns]

# save final results to folder
final_results.to_csv('final_results.csv', index=False)

### Save Results

In [ ]:
#final_results.to_csv('final_results.csv', index=False)

***
## Evaluation

In [ ]:
final_results = pd.read_csv('final_results.csv')

### Average RMSE for each NN Architecture

In [ ]:
final_results.groupby(['model'])['rmse'].mean().reset_index()

### Average RMSE for each Region and Property Type

In [ ]:
rmses = final_results.groupby(['property', 'region'])['rmse'].mean().reset_index()
rmses = round(pd.pivot_table(rmses, values = 'rmse', index='region', columns = 'property').reset_index(), 4)
rmses['Mean'] = round(rmses.mean(axis=1), 4)
rmses.loc[5] = rmses.mean(axis=0)
rmses.loc[5]['region'] = 'Mean'
rmses

### Kupiec Test Results

In [ ]:
round(final_results.groupby(['region'])[['act_pval', 'pred_pval']].mean().reset_index(), 4)

In [ ]:
final_results.groupby(['property'])[['act_pval', 'pred_pval']].mean().reset_index()

#### Further EDA

In [ ]:
def get_summary(df):
    summ = df.describe()[regions]
    skew = pd.DataFrame(df.skew()).T[regions]
    skew = skew.rename(index={0:'skewness'})
    summ.loc["skewness"] = skew.loc['skewness']
    kurt = pd.DataFrame(df.kurt()).T[regions]
    kurt = kurt.rename(index={0:'kurtosis'})
    summ.loc["kurtosis"] = kurt.loc['kurtosis']
    summ = summ.T
    summ = summ[['count', 'min', 'mean', 'max', 'std', 'skewness', 'kurtosis']]
    summ['count'] = pd.to_numeric(summ['count'], downcast='integer')
    
    return round(summ, 3).reset_index()